In [20]:
from wget import download

articles = download("https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip")


In [21]:
import zipfile
with zipfile.ZipFile('new_articles.zip', 'r') as zip_ref:
    zip_ref.extractall("data")


In [25]:
# import shutil
# shutil.unpack_archive('E:\\GenAI\\VectorDB\\new_articles.zip', "data")
import os
os.remove("E:\\GenAI\\VectorDB\\new_articles.zip")

In [26]:
from dotenv import load_dotenv
env_var = load_dotenv()

In [27]:
env_var

True

In [45]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader


#loader =DirectoryLoader("E:\\GenAI\\VectorDB\\data\\",glob= ".\\.text",loader_cls= TextLoader,show_progress= True)
loader = DirectoryLoader("E:\\GenAI\\VectorDB\\data\\", glob="*.txt",loader_cls= TextLoader,show_progress= True)
documents = loader.load()

100%|██████████| 21/21 [00:00<00:00, 1105.93it/s]


In [47]:
len(documents)

21

# create chunks

In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
chunks = text_splitter.split_documents(documents)


In [50]:
len(chunks)

492

In [51]:
chunks[0]

Document(metadata={'source': 'E:\\GenAI\\VectorDB\\data\\05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}, page_content='Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.')

In [53]:
chunks[1]

Document(metadata={'source': 'E:\\GenAI\\VectorDB\\data\\05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}, page_content='Iron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.')

# Creation of embeddings

In [54]:
persist_directory ='db'


In [55]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [59]:
embeddings = OpenAIEmbeddings()
vectordb= Chroma.from_documents(documents = chunks, embedding = embeddings, persist_directory= persist_directory)

In [62]:
vectordb = None

In [63]:
vectordb = Chroma(persist_directory= persist_directory, embedding_function= embeddings)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1964\1535635755.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory= persist_directory, embedding_function= embeddings)


# semantic search

In [66]:
retriever = vectordb.as_retriever()


In [76]:
docs = retriever.get_relevant_documents("what is microsoft?")

In [77]:
len(docs)

4

In [ ]:
# pass this document to LLM to retreive answer

In [78]:
retriever.search_type

'similarity'

In [79]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI


In [80]:
llm = OpenAI()
qa_chain = RetrievalQA.from_chain_type(llm, retriever = retriever, return_source_documents = True)


In [81]:
query = "How much revenue Microsoft raise fund?"
llm_response = qa_chain (query)

In [82]:
print(llm_response)

{'query': 'How much revenue Microsoft raise fund?', 'result': '\nAround $10 billion.', 'source_documents': [Document(metadata={'source': 'E:\\GenAI\\VectorDB\\data\\05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}, page_content='VC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is'), Document(metadata={'source': 'E:\\GenAI\\VectorDB\\data\\05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}, page_content='VC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up

In [90]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\n Sources:')
    for source in llm_response['source_documents']:
        print(source.metadata["source"])

In [91]:
query = "Tell me about Microsoft"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Microsoft is a technology company that creates and sells a variety of software and hardware products, including the Windows operating system, Office 365 suite, and Edge web browser. They have a focus on developing cutting-edge technologies, such as artificial intelligence, and often showcase their latest innovations at their annual developer conference, Microsoft Build. However, they have faced criticism for recent layoffs and the disbandment of their ethics and society team within their AI organization.


 Sources:
E:\GenAI\VectorDB\data\05-04-microsoft-doubles-down-on-ai-with-new-bing-features.txt
E:\GenAI\VectorDB\data\05-04-microsoft-doubles-down-on-ai-with-new-bing-features.txt
E:\GenAI\VectorDB\data\05-04-microsoft-doubles-down-on-ai-with-new-bing-features.txt
E:\GenAI\VectorDB\data\05-04-microsoft-doubles-down-on-ai-with-new-bing-features.txt
